In [1]:
import cv2
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/ai/keras-retinanet-master")
from keras_retinanet.models import load_model

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(device=gpus[1], enable=True)

In [4]:
labels_to_names = {0: 'pack'} 
model = load_model('/ai/keras-retinanet-master/keras_retinanet/bin/inference_model_50.h5', backbone_name='resnet50')

Using TensorFlow backend.


tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

/usr/local/python/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
# load image
# image = read_image_bgr('/ai/keras-retinanet-master/examples/wd.jpg')
# image = cv2.imread('/ai/data/bj/huawei_shop1.jpg')
image_path = '/ai/data/test/'
filenames= os.listdir(image_path)

In [8]:
for filename in filenames:   
    
    image = cv2.imread(image_path + filename)
    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print(filename + " processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < 0.5:
            break

        color = label_color(2)

        b = box.astype(int)
        draw_box(draw, b, color=color,thickness=4)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)

    cv2.imwrite(filename, draw[:,:,::-1])
    # plt.figure(figsize=(15, 15))
    # plt.axis('off')
    # plt.imshow(draw)
    # plt.show()

1.jpg processing time:  0.18030238151550293
2.jpg processing time:  0.17299127578735352
3.jpg processing time:  0.14539837837219238
4.jpg processing time:  0.13512444496154785
5.jpg processing time:  0.12322330474853516
6.jpg processing time:  0.13503551483154297
7.jpg processing time:  0.12358736991882324
8.jpg processing time:  0.12535691261291504
9.jpg processing time:  0.13914132118225098
10.jpg processing time:  0.12727117538452148
11.jpg processing time:  0.14528274536132812
